In [1]:
import openai
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Matplotlib is building the font cache; this may take a moment.


In [2]:
import os

openai.api_key = os.getenv("OPENAI_API_KEY")

In [8]:
with open('contract_case_summaries.json', 'r') as file:
    ground_truth_cases = json.load(file)

In [21]:
ground_truth_df = pd.DataFrame(ground_truth_cases)
ground_truth_df[0:1]['outcome'][0]

{'decision': 'Judgment in favor of Plaintiff Company 4.',
 'legal_reasoning': ['Defendant breached the contract by failing to deliver.',
  'Force majeure was deemed inapplicable.',
  'Losses were foreseeable and directly linked to the breach.'],
 'remedies_awarded': {'damages': '$90000 for lost business opportunities.',
  'attorney_fees': 'Defendant to pay legal costs.',
  'interest': 'Pre-judgment interest applied.'}}

In [28]:
with open('generated_prompts.txt', 'r') as file:
    prompts = file.read().split('---')  # Assuming prompts are separated by '---'

prompts = [prompt.strip() for prompt in prompts if prompt.strip()]


In [31]:
prompts[0]

"Case Title: Case Title 4\n\nYou are a legal expert specializing in U.S. contract law. Based on the following case details, predict the court's decision and provide the legal reasoning."

"Case Title: Case Title 4\n\nYou are a legal expert specializing in U.S. contract law. Based on the following case details, predict the court's decision and provide the legal reasoning."

In [29]:
prompts[1]

"**Case Title**: Case Title 4\n\n**Jurisdiction**: United States District Court for California\n\n### **Facts**\n\n#### **Parties**\n- **Plaintiff**: Plaintiff Company 4, a tech startup.\n- **Defendant**: Defendant Agency 4, a marketing firm.\n\n#### **Contract Details**\n- **Type**: Service Agreement for Marketing Campaign\n- **Key Terms**:\n  - **Delivery Date**: December 5, 2022\n  - **Payment**: $30000 upon delivery of services\n  - **Performance Obligations**: Execution of a targeted advertising campaign for product 4.\n\n#### **Timeline of Events**\n- **July 5, 2022**: Contract signed.\n- **September 14, 2022**: Defendant requests delay; plaintiff refuses.\n- **December 5, 2022**: Defendant fails to deliver services.\n- **December 9, 2022**: Plaintiff files lawsuit.\n\n### **Legal Issues**\n\n#### **Primary Issue(s)**\n1. Did Defendant Agency 4 breach the contract by failing to deliver services by the specified date?\n2. Is Plaintiff Company 4 entitled to damages for losses incur

"**Case Title**: Case Title 4\n\n**Jurisdiction**: United States District Court for California\n\n### **Facts**\n\n#### **Parties**\n- **Plaintiff**: Plaintiff Company 4, a tech startup.\n- **Defendant**: Defendant Agency 4, a marketing firm.\n\n#### **Contract Details**\n- **Type**: Service Agreement for Marketing Campaign\n- **Key Terms**:\n  - **Delivery Date**: December 5, 2022\n  - **Payment**: $30000 upon delivery of services\n  - **Performance Obligations**: Execution of a targeted advertising campaign for product 4.\n\n#### **Timeline of Events**\n- **July 5, 2022**: Contract signed.\n- **September 14, 2022**: Defendant requests delay; plaintiff refuses.\n- **December 5, 2022**: Defendant fails to deliver services.\n- **December 9, 2022**: Plaintiff files lawsuit.\n\n### **Legal Issues**\n\n#### **Primary Issue(s)**\n1. Did Defendant Agency 4 breach the contract by failing to deliver services by the specified date?\n2. Is Plaintiff Company 4 entitled to damages for losses incurred?\n\n#### **Secondary Issue(s)**\n1. Does the defendant's staffing issues qualify as a valid excuse under force majeure?"


In [30]:
prompts[2]

"Please provide:\n\n1. **Decision**: Should the court rule in favor of the plaintiff or the defendant?\n2. **Legal Reasoning**: Explain the reasoning behind the decision, citing relevant legal principles and statutes.\n\n\nCase Title: Case Title 5\n\nYou are a legal expert specializing in U.S. contract law. Based on the following case details, predict the court's decision and provide the legal reasoning."

In [ ]:
prompts[2]